In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ml-quiz1-p5/train.csv
/kaggle/input/2021-ml-quiz1-p5/test.csv
/kaggle/input/2021-ml-quiz1-p5/sample_submit.csv


In [2]:
train_data = pd.read_csv('/kaggle/input/2021-ml-quiz1-p5/train.csv')
test_data = pd.read_csv('/kaggle/input/2021-ml-quiz1-p5/test.csv')
sample = pd.read_csv('/kaggle/input/2021-ml-quiz1-p5/sample_submit.csv')

print(train_data.head())
print(test_data.head())

   id  hour  hour_bef_temperature  hour_bef_precipitation  hour_bef_windspeed  \
0   3    20                  16.3                     1.0                 1.5   
1   6    13                  20.1                     0.0                 1.4   
2   7     6                  13.9                     0.0                 0.7   
3   8    23                   8.1                     0.0                 2.7   
4   9    18                  29.5                     0.0                 4.8   

   hour_bef_humidity  hour_bef_visibility  hour_bef_ozone  hour_bef_pm10  \
0               89.0                576.0           0.027           76.0   
1               48.0                916.0           0.042           73.0   
2               79.0               1382.0           0.033           32.0   
3               54.0                946.0           0.040           75.0   
4                7.0               2000.0           0.057           27.0   

   hour_bef_pm2.5  count  
0            33.0   49.0  
1 

In [3]:
x_data = train_data.drop(['id', 'count'], axis=1)
y_data = train_data['count']
test_data = test_data.drop('id', axis=1)

print(x_data.head())
print(test_data.head())
print(y_data)

   hour  hour_bef_temperature  hour_bef_precipitation  hour_bef_windspeed  \
0    20                  16.3                     1.0                 1.5   
1    13                  20.1                     0.0                 1.4   
2     6                  13.9                     0.0                 0.7   
3    23                   8.1                     0.0                 2.7   
4    18                  29.5                     0.0                 4.8   

   hour_bef_humidity  hour_bef_visibility  hour_bef_ozone  hour_bef_pm10  \
0               89.0                576.0           0.027           76.0   
1               48.0                916.0           0.042           73.0   
2               79.0               1382.0           0.033           32.0   
3               54.0                946.0           0.040           75.0   
4                7.0               2000.0           0.057           27.0   

   hour_bef_pm2.5  
0            33.0  
1            40.0  
2            19.0  


In [4]:
# split
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size = 0.3, random_state=123)
print(x_train.shape)
print(x_val.shape)

(700, 9)
(300, 9)


In [5]:
# regression -knn
from sklearn.neighbors import KNeighborsRegressor
reg = KNeighborsRegressor(weights = 'distance')

reg.fit(x_train, y_train)
val_pred = reg.predict(x_val)

from sklearn.metrics import mean_absolute_error
print(mean_absolute_error(y_val, val_pred))

51.002961711682254


In [6]:
# scaler
from sklearn.preprocessing import StandardScaler
scaler1 = StandardScaler()
x_train_std = scaler1.fit_transform(x_train)
x_val_std = scaler1.transform(x_val)

# minmax
from sklearn.preprocessing import MinMaxScaler
scaler2 = MinMaxScaler()
x_train_nm = scaler2.fit_transform(x_train)
x_val_nm = scaler2.transform(x_val)

In [7]:
reg.fit(x_train_std, y_train)
val_pred_std = reg.predict(x_val_std)
print(mean_absolute_error(y_val, val_pred_std))

reg.fit(x_train_nm, y_train)
val_pred_nm = reg.predict(x_val_nm)
print(mean_absolute_error(y_val, val_pred_nm))

37.553667144694955
34.93923664097536


In [8]:
# scaler2
x_data_nm = scaler2.fit_transform(x_data)
test_data_nm = scaler2.transform(test_data)

reg.fit(x_data_nm, y_data)
pred = reg.predict(test_data_nm)

In [9]:
sample['count'] = pred
sample

,id,count
0,0,27.524581
1,1,246.374718
2,2,139.152676
3,3,29.656970
4,4,27.251716
...,...,...
322,322,26.603718
323,323,27.882652
324,324,98.026610
325,325,207.576994


In [10]:
sample.to_csv('submit.csv', header=True, index=False)